In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
# Make the graphs a bit prettier, and bigger
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_columns', 60)
# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

In [2]:
df = pd.DataFrame(pd.read_excel('D:/p/diag.xlsx'))

In [3]:
df.columns = [ 'order', 'count', 'diag', 'icd', 'note']
df = df.loc[0:,['count', 'diag']]

In [4]:
df.head()

,count,diag
0,8666,窦性心律
1,7058,冠状动脉粥样硬化性心脏病
2,6159,脂代谢紊乱
3,6084,PCI术后
4,5879,2型糖尿病


## CathPCI定义
1. No symptom, no angina
2. Symptom unlikely to be ischemic
3. Stable angina
4. Unstable angina
5. Non-STEMI
6. ST-Elevation MI (STEMI) or equivalent

## 我们的定义
1. 冠心病（未分具体类型）：p1
2. 心绞痛（未分类型）：p2
3. 稳定性心绞痛：p3
4. 不稳定性心绞痛：p4
5. 非ST段抬高型心肌梗死：p5
6. 急性ST段抬高型心肌梗死：p6
7. 急性心肌梗死（未分类型）：p7
8. 急性冠脉综合征（未分类型）：p8
9. 可疑的诊断：p9

In [5]:
def judge_rule(diag):
    p1 = u'冠.*心.*病' #冠状动脉粥样硬化性心脏病，未具体分类，'CHD'
    p2 = u'.*心绞.' #心绞痛，未分类型，'angina'
    p31 = u'(.*稳.*心.*)|(.*初.*劳.*心.*)'
    p32 = u'(不稳)' #稳定性心绞痛，p31+!p32，'stable angina'
    p4 = u'(不稳)|(.*恶.*劳.*心.*)' #不稳定性心绞痛，p4, 'unstable angina'
    p51 = u'(非ST)' 
    p52 = u'(陈旧)' #非ST段抬高型心肌梗死：p5, NSTEMI
    p61 = u'(ST)'
    p62 = u'(陈旧)|(非)|(GIST)|(Stanford)|(Ebstein)' # 急性ST段抬高型心肌梗死：p6，STEMI
    p9 = u'？|\?|可能|不除外|待排' #不确定的诊断
    result = str("")
    if re.search(p1, diag):
        result = result + 'CHD;'
    if re.search(p2, diag):
        result = result + 'angina;'
    if re.search(p31, diag) and re.search(p32, diag)==None:
        result = result + 'stable angina;'
    if re.search(p4, diag):
        result = result + 'unstable angina;'
    if re.search(p51, diag) and re.search(p52, diag)==None:
        result = result + 'Non-STEMI;'
    if re.search(p61, diag) and re.search(p62, diag)==None:
        result = result + 'STEMI;'
    if re.search(p9, diag):
        result = result + '<sus>;'
    return str(result)
df['cad'] = df['diag'].apply(lambda x: judge_rule(x))

In [8]:
df.loc[df.cad.str.contains("Non-STEMI")].diag.drop_duplicates()

63                急性非ST段抬高型心肌梗死
181                 非ST段抬高型心肌梗死
243               急性非ST段抬高性心肌梗死
460                 非ST段抬高性心肌梗死
621                急性非ST段抬高心肌梗死
876           急性非ST段抬高型心肌梗死（前壁）
909             急性前壁非ST段抬高型心肌梗死
1154          急性非ST段抬高型心肌梗死（下壁）
1562        急性非ST段抬高型心肌梗死 PCI术后
1710                 非ST段抬高心肌梗死
1736        急性非ST段抬高性心肌梗死 PCI术后
1804              急性非ST段抬高型心肌梗塞
1830       急性非ST段抬高型心肌梗死（下壁、前壁）
1848       急性非ST段抬高型心肌梗死（前壁、侧壁）
2004              非ST段抬高型急性心肌梗死
2077                急性非ST段抬高型心梗
2395          急性非ST段抬高型心肌梗死（侧壁）
2419            非ST段抬高型心肌梗死（前壁）
2423            急性前壁非ST段抬高性心肌梗死
2434               急性非ST抬高型心肌梗死
2567            急性下壁非ST段抬高型心肌梗死
2830         急性前壁、下壁非ST段抬高型心肌梗死
2978         急性非ST段抬高型心肌梗死（前侧壁）
3381      急性非ST段抬高性心肌梗死（前壁、前间壁）
3534          急性非ST段抬高性心肌梗死（前壁）
3561          急性非ST段抬高性心肌梗死（下壁）
3620           非ST段抬高心肌梗死 PCI术后
3650      急性非ST段抬高型心肌梗死（高侧壁、前壁）
3787                 急性非ST段抬高心梗
3908             急性前壁非ST段抬高心肌梗死
                  ...          
14692   

In [45]:
df[df.cad.str.contains("angina")].groupby("diag").count().sort_values(by="cad",ascending=False)

,count,cad
diag,,
PCI术后 不稳定心绞痛,1,1
微血管病心绞痛不除外,1,1
微血管心绞痛不除外,1,1
微血管心绞痛可能大,1,1
微血管心绞痛可能性大,1,1
微血管性心绞痛,1,1
微血管性心绞痛、变异型心绞痛,1,1
微血管性心绞痛不除外,1,1
微血管性心绞痛可能,1,1


In [75]:
df[df.cad.str.contains("<sus>")].groupby("DIAGNOSECONTENT").count().sort_values(by="cad",ascending=False)

,IN_PATIENT_FLOW,MR_ID,DIAGNOSERANGE,cad
DIAGNOSECONTENT,,,,
不稳定型心绞痛？,61,61,61,61
冠状动脉粥样硬化性心脏病？,19,19,19,19
冠状动脉粥样硬化性心脏病可能性大,12,12,12,12
心律失常？,12,12,12,12
不稳定型心绞痛可能性大,11,11,11,11
冠状动脉粥样硬化性心脏病不除外,11,11,11,11
冠心病？,10,10,10,10
稳定型心绞痛？,9,9,9,9
糖尿病肾病可能性大,8,8,8,8
